In [4]:
pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import pandas as pd
import numpy as np
import requests
import time
from time import sleep
import datetime
from ratelimiter import RateLimiter
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#
airports = pd.read_csv("airports.csv", sep = ";")

In [ ]:
#creation des listes individuels 
np.random.seed(123)

ousmane , abdoul , abdel = np.array_split(np.random.permutation(airports["AirportCode"]), 3)

In [54]:
#collecte de flight_information
def get_flight_information(origins, dates, access_tokens):
    flights = []

    rate_limiter_seconds = RateLimiter(max_calls=4, period=1)  # 4 requêtes par seconde
    rate_limiter_hour = RateLimiter(max_calls=1000, period=3600)  # 1000 requêtes par heure
    api_key_index = 0
    requests_count = 0

    for date in dates:
        for origin in origins:
            for hour in range(24):
                for minute in ["00", "30"]:
                    with rate_limiter_seconds, rate_limiter_hour:
                        if requests_count >= 900:
                            print(f"Changement de clé API : utilisation de la clé {api_key_index + 1}") 
                            api_key_index += 1
                            requests_count = 0
                            if api_key_index >= len(access_tokens):
                                print("Toutes les clés API ont atteint la limite de requêtes.")
                                return pd.DataFrame.from_dict(flights).drop_duplicates().reset_index(drop=True)

                    time_str = f"{hour:02d}:{minute}"
                    datetime = f"{date}T{time_str}"
                    
                    authentification = {"Authorization": "Bearer " + access_tokens[api_key_index]}

                    try:
                        response = requests.get(
                            ("https://api.lufthansa.com/v1/operations/customerflightinformation/departures/{origin}/{datetime}"
                             "?limit=100&offset=0"
                            ).format(origin=origin, datetime=datetime),
                            headers=authentification, timeout=20).json()
                        
                        if response['FlightInformation']['Meta'].get('TotalCount') == 1 :


                            flight = response['FlightInformation']['Flights']['Flight']
                            departure = flight.get('Departure', {})
                            arrival = flight.get('Arrival',{})
                            operatingCarrier = flight.get('OperatingCarrier')
                            equipment = flight.get('Equipment')
                            status = flight.get('Status')

                            row = {
                                    'DpAirportCode': departure.get('AirportCode'),
                                    'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                    'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                    'DpActualDate': departure.get('Actual', {}).get('Date'),
                                    'DpActualTime': departure.get('Actual', {}).get('Time'),
                                    'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                    'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                    'DpStatusCode': departure.get('Status', {}).get('Code'),
                                    'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                    'ArrAirportCode':arrival.get('AirportCode', {}),
                                    'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                    'ArrpScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                    'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                    'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                    'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                    'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                    'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                    'ArrStatusDescription': arrival.get('Status', {}).get('Description'),                             
                                    'AirlineID': operatingCarrier.get('AirlineID'),
                                    'FlightNumber': operatingCarrier.get('FlightNumber'),
                                    'AircraftCode': equipment.get('AircraftCode'),
                                    'StatusCode' : status.get('Code'),
                                    'StatusDescription' : status.get('Description')

                                                    }
                            flights.append(row)
                            requests_count += 1

                        else :

                            for flight in response['FlightInformation']['Flights']['Flight']:
                                # Extraction des données du vol 
                                departure = flight.get('Departure', {})
                                arrival = flight.get('Arrival',{})
                                operatingCarrier = flight.get('OperatingCarrier')
                                equipment = flight.get('Equipment')
                                status = flight.get('Status')


                                row = {
                                    'DpAirportCode': departure.get('AirportCode'),
                                    'DpScheduledDate': departure.get('Scheduled', {}).get('Date'),
                                    'DpScheduledTime': departure.get('Scheduled', {}).get('Time'),
                                    'DpActualDate': departure.get('Actual', {}).get('Date'),
                                    'DpActualTime': departure.get('Actual', {}).get('Time'),
                                    'DpTerminalName': departure.get('Terminal', {}).get('Name'),
                                    'DpTerminalGate': departure.get('Terminal', {}).get('Gate'),
                                    'DpStatusCode': departure.get('Status', {}).get('Code'),
                                    'DPStatusDescription': departure.get('Status', {}).get('Description'),
                                    'ArrAirportCode':arrival.get('AirportCode', {}),
                                    'ArrScheduledDate': arrival.get('Scheduled', {}).get('Date'),
                                    'ArrpScheduledTime': arrival.get('Scheduled', {}).get('Time'),
                                    'ArrActualDate':arrival.get('Actual', {}).get('Date'),
                                    'ArrActualTime':arrival.get('Actual', {}).get('Time'),
                                    'ArrTerminalName':arrival.get('Terminal', {}).get('Name'),
                                    'ArrTerminalGate':arrival.get('Terminal', {}).get('Gate'),
                                    'ArrStatusCode': arrival.get('Status', {}).get('Code'),
                                    'ArrStatusDescription': arrival.get('Status', {}).get('Description'),                             
                                    'AirlineID': operatingCarrier.get('AirlineID'),
                                    'FlightNumber': operatingCarrier.get('FlightNumber'),
                                    'AircraftCode': equipment.get('AircraftCode'),
                                    'StatusCode' : status.get('Code'),
                                    'StatusDescription' : status.get('Description')

                                }

                                flights.append(row)

                            requests_count += 1

                   

                    except requests.exceptions.ReadTimeout:
                        print(f"ReadTimeout error occurred for flight origine {origin}, time {time_str}. Waiting for 5 seconds before continuing.")
                        sleep(5)  # Pause de 5 secondes
                        continue  # continuer

                    except Exception as e:
                        print(f"no flight from origin {origin}, time {time_str}: {e}")

                    

    flight_information_departures = pd.DataFrame.from_dict(flights).reset_index(drop = True)
    flight_information_departures = flight_information_departures.drop_duplicates().reset_index(drop = True)
    flight_information_departures.insert(0, 'FlightId', flight_information_departures['AirlineID'] + flight_information_departures['FlightNumber'].astype(str))
    return flight_information_departures

In [55]:
origins = ousmane
dates = ["2023-05-29"]
access_tokens = ["cle" , "cle",
                 "cle" , "cle",
                 "cle" , "cle",
                 "cle" , "cle"]

flight_information_departures = get_flight_information(origins, dates, access_tokens)
flight_information_departures.to_csv("/content/drive/MyDrive/customer_flight_information_departures_29_ousmane.csv", sep = ";", index=False)
display(flight_information_departures)

no flight from origin ORY, time 00:00: 'FlightInformation'
no flight from origin ORY, time 00:30: 'FlightInformation'
no flight from origin ORY, time 01:00: 'FlightInformation'
no flight from origin ORY, time 01:30: 'FlightInformation'
no flight from origin ORY, time 02:00: 'FlightInformation'
no flight from origin ORY, time 02:30: 'FlightInformation'
no flight from origin ORY, time 03:00: 'FlightInformation'
no flight from origin ORY, time 03:30: 'FlightInformation'
no flight from origin ORY, time 04:00: 'FlightInformation'
no flight from origin ORY, time 04:30: 'FlightInformation'
no flight from origin ORY, time 05:00: 'FlightInformation'
no flight from origin ORY, time 05:30: 'FlightInformation'
no flight from origin ORY, time 10:00: 'FlightInformation'
no flight from origin ORY, time 10:30: 'FlightInformation'
no flight from origin ORY, time 11:00: 'FlightInformation'
no flight from origin ORY, time 11:30: 'FlightInformation'
no flight from origin ORY, time 12:00: 'FlightInformatio

,FlightId,DpAirportCode,DpScheduledDate,DpScheduledTime,DpActualDate,DpActualTime,DpTerminalName,DpTerminalGate,DpStatusCode,DPStatusDescription,...,ArrActualTime,ArrTerminalName,ArrTerminalGate,ArrStatusCode,ArrStatusDescription,AirlineID,FlightNumber,AircraftCode,StatusCode,StatusDescription
0,CL2213,ORY,2023-05-29,09:35,2023-05-29,09:42,1,A19,DP,Flight Departed,...,11:03,2,T02S,LD,Flight Landed,CL,2213,CR9,LD,Flight Landed
1,EN8363,ORY,2023-05-29,19:15,2023-05-29,19:15,None,A21,DP,Flight Departed,...,20:43,2,K21,LD,Flight Landed,EN,8363,E95,LD,Flight Landed
2,LH172,FRA,2023-05-29,06:15,None,None,1,A20,NO,No Status,...,None,1,A03,NO,No Status,LH,172,319,CD,Flight Cancelled
3,LH002,FRA,2023-05-29,06:30,2023-05-29,06:30,1,A18,DP,Flight Departed,...,07:31,2,None,LD,Flight Landed,LH,002,32A,LD,Flight Landed
4,LH1004,FRA,2023-05-29,06:40,None,None,1,A22,NO,No Status,...,None,None,None,NO,No Status,LH,1004,32A,CD,Flight Cancelled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,LH576,FRA,2023-05-29,22:15,2023-05-29,22:24,1,Z66,DP,Flight Departed,...,09:56,None,None,LD,Flight Landed,LH,576,343,LD,Flight Landed
488,LH1402,FRA,2023-05-29,22:15,2023-05-29,22:34,1,A14,DP,Flight Departed,...,23:37,2,None,LD,Flight Landed,LH,1402,321,LD,Flight Landed
489,2A3496,FRA,2023-05-29,22:21,2023-05-29,22:31,Station,T06,DP,Flight Departed,...,23:59,None,None,LD,Flight Landed,2A,3496,TRN,LD,Flight Landed
490,2A3632,FRA,2023-05-29,22:27,2023-05-29,22:22,Station,T06,DP,Flight Departed,...,23:37,None,None,LD,Flight Landed,2A,3632,TRN,LD,Flight Landed
